In [ ]:
# To do: #

# Find out coefs/features. Figure out how to get model into live algo.
# Figure out how to parallelize this, and do it with all top stocks.
# Try this with out-of-the-way shit.
# Try with crypto.
# Cross-reference with classifier loop.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [7]:
import pandas as pd
import numpy as np
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import timedelta
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVR

from sklearn.model_selection import train_test_split

# Set params here

In [15]:
stocks = ['CHK', 'IGC', 'WFT', 'RAD', 'SHLD', 'JCP', 'ABEV']

start = '2000-1-1'
end = '2018-8-1'

rolling_avg = [5, 10, 50, 100]

forward_rets = [20]

# Loop

In [16]:
results = []

for stock in stocks:
    
    print(stock)
    
    try:
        data = yf.download(stock, start, end)
    
        data['pct_change'] = data['Adj Close'].pct_change()

        data['htc'] = (data['High'] - data['Close']) / (data['High'] - data['Low'])
        data['ltc'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])

        ########################
        ### Rolling averages ###
        ########################

        for i in rolling_avg:

            # ema, dist from ema

            ema = data['Close'].ewm(span=i, min_periods=i, adjust=False).mean()
            data['ema_' + str(i)] = ema
            data['df_ema_' + str(i)] = (data['Close'] - ema) / data['Close']

            # dist (mean, std, skew)

            data['change_mean_' + str(i)] = data['pct_change'].ewm(span=i, min_periods=i, adjust=False).mean()
            data['change_std_' + str(i)] = data['pct_change'].ewm(span=i, min_periods=i, adjust=False).std()
            data['change_skew_' + str(i)] = data['pct_change'].rolling(window=i).skew()

            # add ratio dists (mean, std, skew) here

            data['htc_' + str(i)] = data['htc'].ewm(span=i, min_periods=i, adjust=False).mean()
            data['ltc_' + str(i)] = data['ltc'].ewm(span=i, min_periods=i, adjust=False).mean()

        data = data.fillna(0)

        #######################
        ### Forward returns ###
        #######################

        for forward in forward_rets:

            #print('Getting targets for ' + str(forward) + ' days...')

            # for the date, get the forward

            ret_list = []

            for date in range(len(data)):

                # get the return amt close of date+foward

                try:
                    forward_ret = data.iloc[date+forward]['Adj Close']
                    ret_list.append((forward_ret - data.iloc[date]['Adj Close']) / data.iloc[date]['Adj Close'])
                except:
                    ret_list.append(0)

            data['fwd_ret_' + str(forward)] = ret_list

        ###################
        ### Set cutoffs ###
        ###################

        front_cutoff = max(rolling_avg)
        back_cutoff = max(forward_rets)
        data = data[front_cutoff:-back_cutoff]

        ############################
        ### Features and targets ###
        ############################

        features = [x for x in data.columns[5:] if 'fwd' not in x]
        targets = [x for x in data.columns if 'fwd' in x]

        ########################
        ### Get the results! ###
        ########################

        for target in targets:

            try:

                print('Getting ' + str(target) + ' results...')

                X = data[features]
                y = data[target]

                X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

                pf = PolynomialFeatures(include_bias=False) 
                X_poly_train = pf.fit_transform(X_train)         
                cols = pf.get_feature_names(features)
                train_df = pd.DataFrame(X_poly_train, columns=cols)

                pf = PolynomialFeatures(include_bias=False)
                X_poly_test = pf.fit_transform(X_test)    
                test_df = pd.DataFrame(X_poly_test, columns=cols)

                ss = StandardScaler()
                train_scaled = pd.DataFrame(ss.fit_transform(train_df), columns=train_df.columns)

                ss = StandardScaler()
                test_scaled = pd.DataFrame(ss.fit_transform(test_df), columns=train_df.columns)

                rf = RandomForestRegressor()
                rf.fit(train_scaled, y_train)
                rf_res = {
                    'stock': stock,
                    'model': 'RandomForest',
                    'train_score': rf.score(train_scaled, y_train),
                    'test_score': rf.score(test_scaled, y_test),
                    'target': target
                }
                results.append(rf_res)
                
#                 svr = SVR() # kernel
#                 svr.fit(train_scaled, y_train)
#                 svr_res = {
#                     'stock': stock,
#                     'model': 'SVR',
#                     'train_score': svr.score(train_scaled, y_train),
#                     'test_score': svr.score(test_scaled, y_test),
#                     'target': target
#                 }
#                 results.append(svr_res)
                
#                 svr = SVR(kernel='linear') # kernel
#                 svr.fit(train_scaled, y_train)
#                 svr_res_l = {
#                     'stock': stock,
#                     'model': 'SVR_linear',
#                     'train_score': svr.score(train_scaled, y_train),
#                     'test_score': svr.score(test_scaled, y_test),
#                     'target': target
#                 }
#                 results.append(svr_res_l)
                
#                 svr = SVR(kernel='poly') # kernel
#                 svr.fit(train_scaled, y_train)
#                 svr_res_p = {
#                     'stock': stock,
#                     'model': 'SVR_poly',
#                     'train_score': svr.score(train_scaled, y_train),
#                     'test_score': svr.score(test_scaled, y_test),
#                     'target': target
#                 }
#                 results.append(svr_res_p)

                gbc = GradientBoostingRegressor()
                gbc.fit(train_scaled, y_train)
                gbc_res = {
                    'stock': stock,
                    'model': 'GradientBoost',
                    'train_score': gbc.score(train_scaled, y_train),
                    'test_score': gbc.score(test_scaled, y_test),
                    'target': target
                }
                results.append(gbc_res)

                ada = AdaBoostRegressor()
                ada.fit(train_scaled, y_train)
                ada_res = {
                    'stock': stock,
                    'model': 'AdaBoost',
                    'train_score': ada.score(train_scaled, y_train),
                    'test_score': ada.score(test_scaled, y_test),
                    'target': target
                }
                results.append(ada_res)

                dt = DecisionTreeRegressor()
                dt.fit(train_scaled, y_train)
                dt_res = {
                    'stock': stock,
                    'model': 'DecisionTree',
                    'train_score': dt.score(train_scaled, y_train),
                    'test_score': dt.score(test_scaled, y_test),
                    'target': target
                }
                results.append(dt_res)

                bag = BaggingRegressor()
                bag.fit(train_scaled, y_train)
                bag_res = {
                    'stock': stock,
                    'model': 'Bagging',
                    'train_score': bag.score(train_scaled, y_train),
                    'test_score': bag.score(test_scaled, y_test),
                    'target': target
                }
                results.append(bag_res)

            except:
                print('Exception w/results loop: ' + str(stock) + ' ' + str(target))
                
                results = pd.DataFrame(results)
                
                results.to_csv('results_.csv')
                
    except:
        print('Exception w/YF downloader: ' + str(stock))

results = pd.DataFrame(results)

CHK
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
IGC
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
WFT
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
RAD
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
SHLD
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
JCP
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...
ABEV
[*********************100%***********************]  1 of 1 downloaded
Getting fwd_ret_20 results...


In [17]:
# results = pd.DataFrame(results)
results.sort_values(by='test_score', ascending=False)

,model,stock,target,test_score,train_score
0,RandomForest,CHK,fwd_ret_20,0.776285,0.956274
4,Bagging,CHK,fwd_ret_20,0.770463,0.956780
15,RandomForest,RAD,fwd_ret_20,0.720245,0.954543
19,Bagging,RAD,fwd_ret_20,0.690707,0.957634
25,RandomForest,JCP,fwd_ret_20,0.645485,0.944911
16,GradientBoost,RAD,fwd_ret_20,0.643746,0.743917
29,Bagging,JCP,fwd_ret_20,0.637795,0.948005
14,Bagging,WFT,fwd_ret_20,0.636579,0.942924
1,GradientBoost,CHK,fwd_ret_20,0.630283,0.744982
10,RandomForest,WFT,fwd_ret_20,0.620874,0.940030
